## Background / Motivation
Education is fundamental to one's own growth and that of society in its entirety as it equips individuals with the information, abilities, and morals they'll need to thrive in modern life. Certainly not always, but quite frequently. Gains in income, decreased poverty, and increased productivity all point to it as a major factor in the expansion of the economy and the improvement of social conditions.

Our research will look into what role personal attributes play in explaining academic success or failure. 

## Problem statement 

Describe your problem statement. Articulate your objectives using absolutely no jargon. Interpret the problem as inference and/or prediction.

## Data sources
The information was obtained from [Kaggle](https://www.kaggle.com/datasets/dipam7/student-grade-prediction). In reality, though, the data originates from the UCL Machine Learning Repository and the [University of Minho in Portugal](https://pcortez.dsi.uminho.pt/). The data set contains student demographic and performance statistics from two Portuguese secondary schools. And we intend to identify the specific characteristics (which will serve as our predictors) that influence the final G3 grade for the year (outcome variable).

## Stakeholders
A wide variety of stakeholders, such as parents, students, professionals working in the education business, economists, and policymakers, could perhaps have an interest in our projects. It is possible that through examining the dataset and gaining insights, we can discover ways in which school curricula can be improved as well as methods by which policymakers and economists can provide better support for future generations.

## Data quality check ( Julia?)

## Data preparation

In a tabular form, show the distribution of values of each variable used in the analysis - for both categorical and continuous variables. Distribution of a categorical variable must include the number of missing values, the number of unique values, the frequency of all its levels. If a categorical variable has too many levels, you may just include the counts of the top 3-5 levels. 

If the tables in this section take too much space, you may put them in the appendix, and just mention any useful insights you obtained from the data quality check that helped you develop the model or helped you realize the necessary data cleaning / preparation.

Were there any potentially incorrect values of variables that required cleaning? If yes, how did you clean them? 

Did you do any data wrangling or data preparation before the data was ready to use for model development? Did you create any new predictors from exisiting predictors? For example, if you have number of transactions and spend in a credit card dataset, you may create spend per transaction for predicting if a customer pays their credit card bill. Mention the steps at a broad level, you may put minor details in the appendix. Only mention the steps that ended up being useful towards developing your final model(s).


It is important to note that the data used in this analysis has no missing values. However, this may not reflect the typical cleanliness and completeness of real-world data sets, which often contain missing or inconsistent values and require more extensive cleaning and preparation. In future work, it will be important to take these considerations into account and apply appropriate data cleaning and preparation techniques to ensure the validity of the results.

In the data preparation phase, several steps were taken to transform and prepare the data for analysis. The following is a summary of the steps taken:

- Identification of categorical variables: All columns in the data frame were reviewed to identify the categorical variables. A loop comprehension was used to find the number of unique values corresponding to each column, and this information was used to decide on the data processing approach.
- Conversion of yes-no variables to binary variables: The code created a dictionary for binary mapping and applied it to all columns in the data frame that contained 'yes' or 'no' as the response.
- Transformation of predictors with 2 unique values: The code transformed predictors with two unique values into binary variables by mapping them to 0 or 1. This was done for variables such as "school", "sex", "famsize", "address", and "Pstatus".
- Creation of new predictors: To handle variables with more than two unique values, the code created dummy variables using the "get_dummies" function in pandas. The dummy variables were created for the "Mjob", "Fjob", "reason", and "guardian" columns. The original columns were then dropped, and the dummy variables were concatenated with the original data frame.
- Combination of correlated predictors: The code combined the "Dalc" and "Walc" columns into a single "Alc" column to reduce correlation between the two variables, and combined the "Fedu" and "Medu" columns into a single "famEdu" column to capture the combined education of both parents. This helped to reduce data redundancy and the noise in the data set, and removed the multicollinearity.
- Conversion of data types: The original data frame, which consisted of both categorical and numerical values, was converted into one that only consisted of numerical data types or uint8. This made it easier and more convenient for later variable selection and model development.

In conclusion, the data cleaning and preparation phase transformed and prepared the data for analysis by converting categorical variables into binary or numerical values, combining correlated predictors, and converting the data types into a more convenient format for analysis. These steps helped to ensure the validity of the results and facilitated later variable selection and model development.

## Exploratory data analysis

Put the relevant EDA here (visualizations, tables, etc.) that helped you figure out useful predictors for developing the model(s). Only put the EDA that ended up being useful towards developing your final model(s). 

List the insights (as bullet points) you got from EDA that ended up being useful towards developing your final model. 

Again, if there are too many plots / tables, you may put them into appendix, and just mention the insights you got from them.

In the exploratory data analysis phase, the response variable "G3 grade" was transformed into a binary variable by setting a threshold that represented whether a student passed the exam or not. The grade distribution was then plotted to get a rough idea of the number of students in each interval.

The relationship between the transformed grade and the four most important predictors, "failures", "absences", "alcohol", and "famEdu", was then explored through lineplots. This revealed a significant effect of "failures" on the grade, with students who had more absences tending to have lower grades. Additionally, box plots were used to visualize the impact of other categorical predictors such as "mother's job", and "father's job" on the final grade.

While the exploratory data analysis provided valuable insights into the relationships between the predictors and the response, it is important to acknowledge its limitations. The scatter plots showed a general downward or upward trend, but the line tended to be uneven with sharp corners, making it difficult to draw definitive conclusions. In cases where the trend was not very smooth, it was difficult to determine the exact relationship between the predictors and the response. These limitations should be taken into account when interpreting the results of the exploratory data analysis and when making decisions about model development

## Approach

What kind of a model (linear / logistic / other) did you use? What performance metric(s) did you optimize and why?

Is there anything unorthodox / new in your approach? 

What problems did you anticipate? What problems did you encounter? Did the very first model you tried work? 

Did your problem already have solution(s) (posted on Kaggle or elsewhere). If yes, then how did you build upon those solutions, what did you do differently? Is your model better as compared to those solutions in terms of prediction / inference?

**Important: Mention any code repositories (with citations) or other sources that you used, and specifically what changes you made to them for your project.**

---
During Data Exploration, we realized it will remain difficult for us to  predicting a score from 0-20 using predictor variables that are (mostly) binary in value. This can be problematic because binary variables typically have a non-linear relationship with the outcome variable, and it can be difficult to predict a continuous score with such data. Moreover, using all categorical close-to-binary data for a prediction problem from a scale of 0-20 is a bad idea because it can result in inaccurate predictions and may not capture the nuances of the data. Therefore, we chose to approach the research question with classification method. 

## Developing the model

Explain the steps taken to develop and improve the base model - informative visualizations / addressing modeling assumption violations / variable transformation / interactions / outlier treatment / influential points treatment / addressing over-fitting / addressing multicollinearity / variable selection - stepwise regression, lasso, ridge regression). 

Did you succeed in achieving your goal, or did you fail? Why?

**Put the final model equation**.

**Important: This section should be rigorous and thorough. Present detailed information about decision you made, why you made them, and any evidence/experimentation to back them up.**

In this study, a classification approach was used to predict student performance. However, this was not a logistic regression classification problem. Instead, the response was manually categorized by setting a threshold and assigning values greater than the threshold as "pass" and values less than or equal to the threshold as "fail".

This approach is better than converting the response to binary values at the outset because it allows for more nuanced categorization of the data. By setting the threshold and manually categorizing the response, it is possible to take into account the variability in the data and make more informed predictions.

This manual categorization approach is a valid method for solving this type of problem. By setting the threshold and categorizing the response, it is possible to effectively convert the continuous response into a categorical response, which can then be used for classification. This approach allows for a more nuanced understanding of the data and provides a more accurate picture of the relationships between the predictors and the response.

#### Base Model Development
Several techniques were used to develop and evaluate the model for the initial and variable selection phase and base model development. The following is a summary of the techniques used:

- K-Fold cross-validation and train-test split: K-Fold cross-validation and train-test split were used to assess the model's accuracy. In K-Fold cross-validation, the data was divided into K equal parts, and the model was trained and tested K times, with each part being used as the test set once. In the train-test split, the data was divided into a training set and a validation set, and the model was trained on the training set and tested on the validation set.

- Decision tree search: A decision tree was built for numerical feature selection. We used `DecisionTreeRegressor` as opposed to `DecisionTreeClassifier` as we were treating the problem as a prediction problem on the first hand, followed by manual classification based on the pass/fail threshold. Variables `failures` and `absences` were identified as the two most important features.

- Chi-square test variable selection: The chi-square test was used to select the most important categorical predictors by assessing the dependence between the categorical predictors and the target variable. `schoolsup` (extra educational support( and parents' job (mother's job `Mjob` and father's job `Fjob`) were important as relatively important predictors.

Given the limitations of a non-linear regressor (`DecisionTreeRegressor`) in identifying important features for linear regression, more than one decision tree was built to analyze and identify the important features. While the selected features often vary,  the most important factors kept at each tree's top branch - features consistently selected as the most important ones regardless of trees built - were `failures` and `absences`. With this information, the base model was developed using only these 2 most important features, along with their interaction terms.

$$G3 \sim absences$$
$$G3 \sim failures$$
$$G3 \sim absences * failures$$

Three models was fitted using these three predictors (`failures`, `absences`, `failures * absences`), and a 100% accuracy was achieved on the training and testing data. Accuracy was used as the only metric at this stage, as it provides a direct measure of the model's basic performance without giving weight to the cost of different errors. Later, more refined metrics will be used to weigh the priorities and relative costs of different types of errors.

## Limitations of the model with regard to inference / prediction

If it is inference, will the inference hold for a certain period of time, for a certain subset of population, and / or for certain conditions.

If it is prediction, then will it be possible / convenient / expensive for the stakeholders to collect the data relating to the predictors in the model. Using your model, how soon will the stakeholder be able to predict the outcome before the outcome occurs. For example, if the model predicts the number of bikes people will rent in Evanston on a certain day, then how many days before that day will your model be able to make the prediction. This will depend on how soon the data that your model uses becomes available. If you are predicting election results, how many days / weeks / months / years before the election can you predict the results. 

When will your model become too obsolete to be useful?

---
The objective of the research project was to classify the performance of secondary school students in two Portuguese secondary schools   Unfortunately, the study contains a number of inference limitations. First, the dataset is from 2008, therefore it may not reflect the current educational scene. In addition, the study had 395 observations in total, which increases the danger of overfitting and makes it challenging to apply the results to a larger population. In addition, the study was done in Portugal, which may limit its applicability to other nations or areas. 

Future study could benefit from increasing the dataset to include a larger sample size and a broader range of demographic and socioeconomic variables, potentially encompassing a variety of nations. This would allow for more rigorous studies and findings that may be applied to a wider range of population types. It may aid in the identification of gaps in educational results and provide insight into potential interventions that may be implemented to assist underrepresented populations.  Inclusion of a larger range of indicators, such as cultural influences, mental health, and family environment, could further illuminate the intricate relationship between individual determinants and educational results.

## Other sections *(optional)*

You are welcome to introduce additional sections or subsections, if required, to address any specific aspects of your project in detail. For example, you may briefly discuss potential future work that the research community could focus on to make further progress in the direction of your project's topic.

## Conclusions and Recommendations to stakeholder(s)

What conclusions do you draw based on your model? If it is inference you may draw conclusions based on the coefficients, statistical significance of predictors / interactions, etc. If it is prediction, you may draw conclusions based on prediction accuracy, or other performance metrics.

How do you use those conclusions to come up with meaningful recommendations for stakeholders? The recommendations must be action-items for stakeholders that they can directly implement without any further analysis. Be as precise as possible. The stakeholder(s) are depending on you to come up with practically implementable recommendations, instead of having to think for themselves.

If your recommendations are not practically implementable by stakeholders, how will they help them? Is there some additional data / analysis / domain expertise you need to do to make the recommendations implementable? 

Do the stakeholder(s) need to be aware about some limitations of your model? Is your model only good for one-time use, or is it possible to update your model at a certain frequency (based on recent data) to keep using it in the future? If it can be used in the future, then for how far into the future?


---
Notwithstanding the limits of the current data set, the findings show the importance of attendance and previous assessments in predicting student achievement, which can be utilized to drive policy decisions and improve student educational outcomes.

## GitHub and individual contribution {-}

[Github link](https://github.com/Juliaaaachu/LinearModel_Stat303-2)

<html>
<style>
table, td, th {
  border: 1px solid black;
}

table {
  border-collapse: collapse;
  width: 100%;
}

th {
  text-align: left;
}
    

</style>
<body>

<h2>Individual contribution</h2>

<table style="width:100%">
     <colgroup>
       <col span="1" style="width: 15%;">
       <col span="1" style="width: 20%;">
       <col span="1" style="width: 50%;">
       <col span="1" style="width: 15%;"> 
    </colgroup>
  <tr>
    <th>Team member</th>
    <th>Contributed aspects</th>
    <th>Details</th>
    <th>Number of GitHub commits</th>
  </tr>
  <tr>
    <td>Julia Chu</td>
    <td>EDA, Base Model, Github Management</td>
    <td> conducted initial exploratory data analysis along with visualization that provided insights for model development and variable transformation. </td>
    <td>12</td>
  </tr>
  <tr>
    <td>V S</td>
    <td>Assumptions and interactions</td>
    <td>Checked and addressed modeling assumptions and identified relevant variable interactions.</td>
    <td>120</td>
  </tr>
    <tr>
    <td>Sankaranarayanan Balasubramanian</td>
    <td>Outlier and influential points treatment</td>
    <td>Identified outliers/influential points and analayzed their effect on the model.</td>
    <td>130</td>    
  </tr>
    <tr>
    <td>Chun-Li</td>
    <td>Variable selection and addressing overfitting</td>
    <td>Performed variable selection on an exhaustive set of predictors to address multicollinearity and overfitting.</td>
    <td>150</td>    
  </tr>
</table>

List the **challenges** you faced when collaborating with the team on GitHub. Are you comfortable using GitHub? 
Do you feel GitHuB made collaboration easier? If not, then why? *(Individual team members can put their opinion separately, if different from the rest of the team)*

## References {-}

[1] P. Cortez and A. Silva. Using Data Mining to Predict Secondary School Student Performance. In A. Brito and J. Teixeira Eds., Proceedings of 5th FUture BUsiness TEChnology Conference (FUBUTEC 2008) pp. 5-12, Porto, Portugal, April, 2008, EUROSIS, ISBN 978-9077381-39-7.

[2] Dipam7. (2021). Student Grade Prediction [Data file]. Kaggle. https://www.kaggle.com/dipam7/student-grade-prediction

## Appendix {-}

You may put additional stuff here as Appendix. You may refer to the Appendix in the main report to support your arguments. However, the appendix section is unlikely to be checked while grading, unless the grader deems it necessary.